In [1]:
#!pip install pandas pandas-ta plotly ZigZag scipy TA-Lib tensorflow scikit-learn tqdm

In [1]:
import tensorflow as tf
import os

import talib
import numpy as np
import pandas as pd
import pandas_ta as ta
from sklearn.preprocessing import scale
from tqdm import tqdm
import random

2022-06-10 12:47:48.508066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-10 12:47:48.508120: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Переменные

In [74]:
NAME = 'Merge_order_book_with_candlestick_data'
TIME_INTERVAL = 60
TAKE_STEPS_TO_WATCH = 256
INPUT_WIDTH = 512
LSTM_INPUT_WIDTH = 512
TAKE_PROFIT_PERCENT = 2
STOP_LOSS_PERCENT = 1
CURRENCY_PAIR = 'btc_usdt'
DROPOUT = 0.01        # 0.1
L2_REGULARIZERS = 0.002 # 0.02
LOOKAHEAD = False
INDICATORS = False

In [75]:
RATES_PATH = f"{os.getcwd()}/../csv"
ORDERBOOK_PATH = f"{os.getcwd()}/../csv/order_book"
EXAMPLE_PATH = f"{os.getcwd()}/checkpoints/"

indicators = ''

if INDICATORS is True:
    indicators = 'indicators'

CHECKPOINT_PATH = f"{EXAMPLE_PATH}/{NAME} {CURRENCY_PAIR.upper()} {TIME_INTERVAL} steps {TAKE_STEPS_TO_WATCH} iw {INPUT_WIDTH} lstm {LSTM_INPUT_WIDTH} tp {TAKE_PROFIT_PERCENT} sl {STOP_LOSS_PERCENT} {indicators}/"

if not os.path.exists(EXAMPLE_PATH):
  os.mkdir(EXAMPLE_PATH)

if not os.path.exists(CHECKPOINT_PATH):
  os.mkdir(CHECKPOINT_PATH)

# Функция индикаторов

In [76]:
#@title
def append_all_indicators(df):

  df = append_candles_indicators(df)
  df = append_overlap_indicators(df)
  df = append_momentum_indicators(df)
  df = append_statistics_indicators(df)
  df = append_trend_indicators(df)
  df = append_aberration_indicators(df)
  df = append_volume_indicators(df)

  return df

def append_candles_indicators(df):

  names = [
           "2crows",
          "3blackcrows",
          "3inside",
          "3linestrike",
          "3outside",
          "3starsinsouth",
          "3whitesoldiers",
          "abandonedbaby",
          "advanceblock",
          "belthold",
          "breakaway",
          "closingmarubozu",
          "concealbabyswall",
          "counterattack",
          "darkcloudcover",
          "doji",
          "dojistar",
          "dragonflydoji",
          "engulfing",
          "eveningdojistar",
          "eveningstar",
          "gapsidesidewhite",
          "gravestonedoji",
          "hammer",
          "hangingman",
          "harami",
          "haramicross",
          "highwave",
          "hikkake",
          "hikkakemod",
          "homingpigeon",
          "identical3crows",
          "inneck",
          "inside",
          "invertedhammer",
          "kicking",
          "kickingbylength",
          "ladderbottom",
          "longleggeddoji",
          "longline",
          "marubozu",
          "matchinglow",
          "mathold",
          "morningdojistar",
          "morningstar",
          "onneck",
          "piercing",
          "rickshawman",
          "risefall3methods",
          "separatinglines",
          "shootingstar",
          "shortline",
          "spinningtop",
          "stalledpattern",
          "sticksandwich",
          "takuri",
          "tasukigap",
          "thrusting",
          "tristar",
          "unique3river",
          "upsidegap2crows",
          "xsidegap3methods"
  ]

  df.ta.cdl_pattern(name=names, append=True)

  return df

def append_momentum_indicators(df):

  df.ta.apo(append=True)
  df.ta.bias(append=True)
  df.ta.bop(append=True)
  df.ta.brar(append=True)
  df.ta.cci(append=True)
  df.ta.cfo(append=True)
  df.ta.cg(append=True)
  df.ta.cmo(append=True)
  df.ta.coppock(append=True)
  df.ta.cti(append=True)
  df.ta.dm(append=True)
  df.ta.er(append=True)
  df.ta.eri(append=True)
  df.ta.fisher(append=True)
  df.ta.inertia(append=True)
  df.ta.kdj(append=True)
  df.ta.kst(append=True)
  df.ta.macd(append=True, )
  df.ta.mom(append=True)
  df.ta.pgo(append=True)
  df.ta.ppo(append=True)
  df.ta.psl(append=True)
  df.ta.pvo(append=True)
  df.ta.qqe(append=True)
  df.ta.roc(append=True)
  df.ta.rsi(append=True)
  df.ta.rsx(append=True)
  df.ta.rvgi(append=True)
  # df.ta.stc(append=True) вываливается ошибка
  df.ta.slope(append=True)
  df.ta.smi(append=True)
  df.ta.squeeze(append=True)
  df.ta.squeeze_pro(append=True)
  df.ta.stoch(append=True)
  df.ta.stochrsi(append=True)
  #df.ta.td_seq(append=True)   # медленный
  df['TD_SEQ_UP'] = 0
  df['TD_SEQ_DN'] = 0
  df.ta.trix(append=True)
  df.ta.tsi(append=True)
  df.ta.uo(append=True)
  df.ta.willr(append=True)

  return df

def append_overlap_indicators(df):

  df.ta.alma(append=True)
  df.ta.dema(append=True)
  df.ta.ema(append=True)
  df.ta.fwma(append=True)
  df.ta.hilo(append=True)
  df.ta.hl2(append=True)
  df.ta.hlc3(append=True)
  df.ta.hma(append=True)
  df.ta.hwma(append=True)
  #df.ta.ichimoku(append=True, lookahead=LOOKAHEAD) # data leak
  #df.ta.jma(append=True)
  df.ta.kama(append=True)
  df.ta.linreg(append=True)
  df.ta.mcgd(append=True)
  df.ta.midpoint(append=True)
  df.ta.midprice(append=True)
  df.ta.ohlc4(append=True)
  df.ta.pwma(append=True)
  df.ta.rma(append=True)
  df.ta.sinwma(append=True)
  df.ta.sma(append=True)
  df.ta.ssf(append=True)
  df.ta.supertrend(append=True)
  df.ta.swma(append=True)
  df.ta.t3(append=True)
  df.ta.tema(append=True)
  df.ta.trima(append=True)
  df.ta.vidya(append=True)
  #df.ta.vwap(append=True) ошибка
  df.ta.vwma(append=True)
  df.ta.wcp(append=True)
  df.ta.wma(append=True)
  df.ta.zlma(append=True)

  return df

def append_statistics_indicators(df):
  df.ta.entropy(append=True)
  df.ta.kurtosis(append=True)
  df.ta.mad(append=True)
  df.ta.median(append=True)
  df.ta.quantile(append=True)
  df.ta.skew(append=True)
  df.ta.stdev(append=True)

  if LOOKAHEAD:
    df.ta.tos_stdevall(append=True) #возможная утечка данных в будущее
  else:
    df['TOS_STDEVALL_LR'] = 0
    df['TOS_STDEVALL_L_1'] = 0
    df['TOS_STDEVALL_U_1'] = 0
    df['TOS_STDEVALL_L_2'] = 0
    df['TOS_STDEVALL_U_2'] = 0
    df['TOS_STDEVALL_L_3'] = 0
    df['TOS_STDEVALL_U_3'] = 0

  df.ta.variance(append=True)
  df.ta.zscore(append=True)

  return df

def append_trend_indicators(df):
  df.ta.adx(append=True)
  df.ta.amat(append=True)
  df.ta.aroon(append=True)
  df.ta.chop(append=True)
  df.ta.cksp(append=True)
  #df.ta.decay(append=True) ошибка
  df.ta.decreasing(append=True)
  df.ta.dpo(append=True, lookahead=LOOKAHEAD) # data leak
  df.ta.increasing(append=True)
  df.ta.long_run(append=True)
  df.ta.psar(append=True)
  df.ta.qstick(append=True)
  df.ta.short_run(append=True)
  df.ta.tsignals(append=True)
  df.ta.ttm_trend(append=True)
  df.ta.vhf(append=True)
  df.ta.vortex(append=True)
  df.ta.xsignals(append=True)

  return df

def append_utility_indicators(df):
  # не работают
  df.ta.above(append=True)
  df.ta.above_value(append=True)
  df.ta.below(append=True)
  df.ta.below_value(append=True)
  df.ta.cross(append=True)

  return df

def append_aberration_indicators(df):
  df.ta.aberration(append=True)
  df.ta.accbands(append=True)
  df.ta.atr(append=True)
  df.ta.bbands(append=True)
  df.ta.donchian(append=True)
  #df.ta.hwc(append=True) ошибка
  df.ta.kc(append=True)
  df.ta.massi(append=True)
  df.ta.natr(append=True)
  df.ta.pdist(append=True)
  df.ta.rvi(append=True)
  df.ta.thermo(append=True)
  df.ta.true_range(append=True)
  df.ta.ui(append=True)

  return df

def append_volume_indicators(df):
  df.ta.ad(append=True)
  df.ta.adosc(append=True)
  df.ta.aobv(append=True)
  df.ta.cmf(append=True)
  df.ta.efi(append=True)
  df.ta.eom(append=True)
  df.ta.kvo(append=True)
  df.ta.mfi(append=True)
  df.ta.nvi(append=True)
  df.ta.obv(append=True)
  df.ta.pvi(append=True)
  df.ta.pvol(append=True)
  df.ta.pvr(append=True)
  df.ta.pvt(append=True)
  df.ta.vp(append=True)

  return df

def append_cycles(df):
  #df.ta.ebsw(append=True) появляется ошибка

  return df

In [77]:
def load_candlesticks():
    file_name = f'{CURRENCY_PAIR}_{TIME_INTERVAL}.csv.zip'

    file_path = f'file://{RATES_PATH}/{file_name}'

    df = pd.read_csv(file_path, decimal='.', keep_default_na=False, encoding = "UTF-8", compression='zip')

    df = df.fillna(0)

    df['datetime'] = pd.to_datetime(df['datetime'])

    df['timestamp'] = df['datetime'].values.astype('int') / 10 ** 9

    df = df.drop_duplicates(subset=['timestamp'])

    df = df.set_index('timestamp')

    return (df.replace((np.inf, -np.inf), np.nan).dropna())

candlesticks_df = load_candlesticks()

In [78]:
candlesticks_df = append_all_indicators(candlesticks_df)

In [79]:
candlesticks_df.head(10)

,datetime,open,close,low,high,volume,CDL_2CROWS,CDL_3BLACKCROWS,CDL_3INSIDE,CDL_3LINESTRIKE,...,PVI_1,PVOL,PVR,PVT,low_close,mean_close,high_close,pos_volume,neg_volume,total_volume
timestamp,,,,,,,,,,,,,,,,,,,,,
1.652887e+09,2022-05-18 15:17:00,29132.87,29116.06,29103.52,29145.24,3.614459,0.0,0.0,0.0,0.0,...,1000.000000,1.052388e+05,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.652887e+09,2022-05-18 15:18:00,29111.64,29068.98,29037.94,29111.64,16.314435,0.0,0.0,0.0,0.0,...,999.838302,4.742440e+05,3.0,-2.638007,NaN,NaN,NaN,NaN,NaN,NaN
1.652887e+09,2022-05-18 15:19:00,29065.84,29055.90,29055.90,29096.61,11.120514,0.0,0.0,0.0,0.0,...,999.838302,3.231165e+05,4.0,-3.138390,NaN,NaN,NaN,NaN,NaN,NaN
1.652887e+09,2022-05-18 15:20:00,29055.54,28983.71,28957.60,29055.54,38.810543,0.0,0.0,0.0,0.0,...,999.589850,1.124874e+06,3.0,-12.780952,NaN,NaN,NaN,NaN,NaN,NaN
1.652887e+09,2022-05-18 15:21:00,28983.71,28927.39,28888.00,28983.71,25.944752,0.0,0.0,0.0,0.0,...,999.589850,7.505140e+05,4.0,-17.822433,NaN,NaN,NaN,NaN,NaN,NaN
1.652887e+09,2022-05-18 15:22:00,28923.14,28901.67,28900.00,28966.31,14.501105,0.0,0.0,0.0,0.0,...,999.589850,4.191061e+05,4.0,-19.111760,NaN,NaN,NaN,NaN,NaN,NaN
1.652887e+09,2022-05-18 15:23:00,28900.00,28920.36,28881.49,28953.36,10.951758,0.0,0.0,0.0,0.0,...,999.589850,3.167288e+05,2.0,-18.403536,NaN,NaN,NaN,NaN,NaN,NaN
1.652887e+09,2022-05-18 15:24:00,28920.04,28948.81,28878.66,28948.83,11.534608,0.0,0.0,0.0,0.0,...,999.688224,3.339132e+05,1.0,-17.268835,NaN,NaN,NaN,NaN,NaN,NaN
1.652888e+09,2022-05-18 15:25:00,28948.81,28986.30,28942.08,29000.00,7.489515,0.0,0.0,0.0,0.0,...,999.688224,2.170933e+05,2.0,-16.298910,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
def load_orderbook():
    file_name = f'order_book_binance_{CURRENCY_PAIR}_{TIME_INTERVAL}.csv.zip'

    file_path = f'file://{ORDERBOOK_PATH}/{file_name}'

    df = pd.read_csv(file_path, decimal='.', keep_default_na=False, encoding = "UTF-8", compression='zip')

    df = df.fillna(0)

    df['datetime'] = pd.to_datetime(df['datetime'])

    df['datetime'] = df['datetime'].dt.floor('T')

    df['timestamp'] = df['datetime'].values.astype('int') / 10 ** 9

    df = df.drop_duplicates(subset=['timestamp'])

    df = df.set_index('timestamp')

    return (df.replace((np.inf, -np.inf), np.nan).dropna())

orderbook_df = load_orderbook()

In [81]:
orderbook_df.head(10)

,datetime,asks,bids
timestamp,,,
1.652954e+09,2022-05-19 09:49:00,243.39134,367.23632
1.652954e+09,2022-05-19 09:50:00,250.75648,367.45799
1.652954e+09,2022-05-19 09:51:00,318.21946,384.56824
1.652954e+09,2022-05-19 09:52:00,333.41531,453.30601
1.652954e+09,2022-05-19 09:53:00,340.42332,454.15463
1.652954e+09,2022-05-19 09:54:00,276.89298,454.26024
1.652954e+09,2022-05-19 09:55:00,333.05469,429.23010
1.652954e+09,2022-05-19 09:56:00,335.29685,459.45611
1.652954e+09,2022-05-19 09:57:00,237.64129,330.27589


In [82]:
candlesticks_df['asks'] = orderbook_df['asks']
candlesticks_df['bids'] = orderbook_df['bids']

candlesticks_df = candlesticks_df[candlesticks_df['asks'].notna()]
candlesticks_df = candlesticks_df[candlesticks_df['bids'].notna()]
df = candlesticks_df

In [83]:
def processing(df, row, take_steps_to_watch):

  index = row.name

  loc = df.index.get_loc(index) + 1

  sample = df.iloc[loc:loc + take_steps_to_watch]

  position = 0

  for index, sample_row in sample.iterrows():
    if sample_row['high'] >= row['long_take_profit_price']:
      return 1
    if sample_row['low'] <= row['long_stop_loss_price']:
      return 0
      
  return position

def append_position(original_df):

  df = original_df.copy()

  df['long_take_profit_price'] = (100 + TAKE_PROFIT_PERCENT) * df['close'] / 100
  df['long_stop_loss_price'] = (100 - STOP_LOSS_PERCENT) * df['close'] / 100

  for index, row in df.iterrows():
    df.at[index, 'position'] = processing(df, row, TAKE_STEPS_TO_WATCH)

  original_df['position'] = df['position']

  return original_df

df = append_position(df)

In [84]:
def append_position(original_df):

  df = original_df.copy()

  df[f'future_high_{TAKE_STEPS_TO_WATCH}'] = df['high'].rolling(TAKE_STEPS_TO_WATCH).max().shift(-TAKE_STEPS_TO_WATCH)
  df[f'future_low_{TAKE_STEPS_TO_WATCH}'] = df['low'].rolling(TAKE_STEPS_TO_WATCH).min().shift(-TAKE_STEPS_TO_WATCH)

  df[f'future_high_{TAKE_STEPS_TO_WATCH}_percentage'] = (df[f'future_high_{TAKE_STEPS_TO_WATCH}'] * 100 / df['close']) - 100
  df[f'future_low_{TAKE_STEPS_TO_WATCH}_percentage'] = 100 - (df[f'future_low_{TAKE_STEPS_TO_WATCH}'] * 100 / df['close'])

  df['long_position'] = 0

  df.loc[(df[f'future_high_{TAKE_STEPS_TO_WATCH}_percentage'] > TAKE_PROFIT_PERCENT) & (df[f'future_low_{TAKE_STEPS_TO_WATCH}_percentage'] < STOP_LOSS_PERCENT), 'long_position'] = 1

  df['short_position'] = 0

  df.loc[(df[f'future_high_{TAKE_STEPS_TO_WATCH}_percentage'] < STOP_LOSS_PERCENT) & (df[f'future_low_{TAKE_STEPS_TO_WATCH}_percentage'] > TAKE_PROFIT_PERCENT), 'short_position'] = 1

  df['position'] = 0

  df.loc[df['long_position'] > 0, 'position'] = 1
  df.loc[df['short_position'] > 0, 'position'] = -1

  original_df['position'] = df['position']

  return original_df

#df = append_position(df)

In [85]:
print("Long trend count:", len(df[df['position'] > 0]))
print("Neutral trend count:", len(df[df['position'] == 0]))
print("Short trend count:", len(df[df['position'] < 0]))

Long trend count: 2508
Neutral trend count: 26575
Short trend count: 0


In [86]:
index = df.columns

features_names_with_position = index.delete([
                                             index.get_loc('datetime'),
                                             index.get_loc('volume'), 
                                             index.get_loc('close'),
                                             index.get_loc('open'), 
                                             index.get_loc('high'), 
                                             index.get_loc('low'),
                                             ])

FEATURES_NAMES = features_names_with_position.delete([features_names_with_position.get_loc('position')]).values

FEATURES_NAMES

array(['CDL_2CROWS', 'CDL_3BLACKCROWS', 'CDL_3INSIDE', 'CDL_3LINESTRIKE',
       'CDL_3OUTSIDE', 'CDL_3STARSINSOUTH', 'CDL_3WHITESOLDIERS',
       'CDL_ABANDONEDBABY', 'CDL_ADVANCEBLOCK', 'CDL_BELTHOLD',
       'CDL_BREAKAWAY', 'CDL_CLOSINGMARUBOZU', 'CDL_CONCEALBABYSWALL',
       'CDL_COUNTERATTACK', 'CDL_DARKCLOUDCOVER', 'CDL_DOJI_10_0.1',
       'CDL_DOJISTAR', 'CDL_DRAGONFLYDOJI', 'CDL_ENGULFING',
       'CDL_EVENINGDOJISTAR', 'CDL_EVENINGSTAR', 'CDL_GAPSIDESIDEWHITE',
       'CDL_GRAVESTONEDOJI', 'CDL_HAMMER', 'CDL_HANGINGMAN', 'CDL_HARAMI',
       'CDL_HARAMICROSS', 'CDL_HIGHWAVE', 'CDL_HIKKAKE', 'CDL_HIKKAKEMOD',
       'CDL_HOMINGPIGEON', 'CDL_IDENTICAL3CROWS', 'CDL_INNECK',
       'CDL_INSIDE', 'CDL_INVERTEDHAMMER', 'CDL_KICKING',
       'CDL_KICKINGBYLENGTH', 'CDL_LADDERBOTTOM', 'CDL_LONGLEGGEDDOJI',
       'CDL_LONGLINE', 'CDL_MARUBOZU', 'CDL_MATCHINGLOW', 'CDL_MATHOLD',
       'CDL_MORNINGDOJISTAR', 'CDL_MORNINGSTAR', 'CDL_ONNECK',
       'CDL_PIERCING', 'CDL_RICKSHAWMAN', 

In [87]:
data = df.loc[:,features_names_with_position]

In [88]:
n = len(data)
max_val_df_length = 2000
max_eval_df_length = 2000
val_length = int(n*0.9) - int(n*0.8)
eval_length = int(n*1) - int(n*0.9)

if val_length > max_val_df_length:
    val_length = max_val_df_length
    
if eval_length > max_eval_df_length:
    eval_length = max_eval_df_length

val_start_index = n - eval_length - val_length
test_start_index = n - eval_length

In [89]:
print(len(data), val_start_index, test_start_index)

29083 25083 27083


In [90]:
train_df = data.iloc[0:val_start_index]
val_df = data.iloc[val_start_index:test_start_index]
test_df = data.iloc[test_start_index:]

test_full_df = df.iloc[test_start_index:]

num_features = data.shape[1]

In [91]:
print(len(data), len(train_df), len(val_df), len(test_df))

29083 25083 2000 2000


In [92]:
def normalization(dataframe, train_mean, train_std):
  position = dataframe.pop('position')
  dataframe = (dataframe - train_mean) / train_std
  dataframe['position'] = position
  return dataframe

In [93]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = normalization(train_df, train_mean, train_std)
val_df = normalization(val_df, train_mean, train_std)
test_df = normalization(test_df, train_mean, train_std)

In [94]:
print(train_mean, train_std)

CDL_2CROWS          -0.035881
CDL_3BLACKCROWS     -0.055815
CDL_3INSIDE          0.115616
CDL_3LINESTRIKE     -0.023921
CDL_3OUTSIDE         0.059801
                      ...    
neg_volume                NaN
total_volume              NaN
asks               354.564047
bids               500.942878
position             0.072479
Length: 263, dtype: float64 CDL_2CROWS           1.893923
CDL_3BLACKCROWS      2.361901
CDL_3INSIDE         11.382514
CDL_3LINESTRIKE      3.993384
CDL_3OUTSIDE        18.741514
                      ...    
neg_volume                NaN
total_volume              NaN
asks               130.459269
bids               498.387881
position             0.259285
Length: 263, dtype: float64


In [95]:
print(len(train_df), len(val_df), len(test_df))

25083 2000 2000


In [96]:
train_df = train_df.fillna(0)
val_df = val_df.fillna(0)
test_df = test_df.fillna(0)

train_df[train_df.isnull().any(axis=1)]
train_df[train_df.isna().any(axis=1)]

,ABER_ATR_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ZG_5_15,ACCBL_20,ACCBM_20,ACCBU_20,AD,ADOSC_3_10,ADX_14,...,ZS_30,asks,bids,high_close,low_close,mean_close,neg_volume,pos_volume,position,total_volume
timestamp,,,,,,,,,,,,,,,,,,,,,


In [97]:
assert train_df.isnull().values.any() == False, 'Тренировочный датафрейм не должен содержать nan значений'
assert val_df.isnull().values.any() == False, 'Датафрейм валидации не должен содержать nan значений'
assert test_df.isnull().values.any() == False, 'Тестовый датафрейм не должен содержать nan значений'

In [98]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None, input_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    self.input_columns = input_columns

    self.input_columns_indices = {name: i for i, name in
                                    enumerate(input_columns)}   

    self.label_columns = label_columns

    self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}                       

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Column indices: {self.column_indices}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}',
        f'Label start: {self.label_start}',
        f'Input width: {self.input_width}',
        f'Label width: {self.label_width}'])

In [99]:
def make_dataset(self, data, shuffle=True, batch_size=32,):
  data = np.array(data, dtype=np.float32)

  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=shuffle,
      batch_size=batch_size)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [100]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [101]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]

  inputs = tf.stack(
        [inputs[:, :, self.column_indices[name]] for name in self.input_columns],
        axis=-1)
  
  labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [102]:
class Model():
  def __init__(self, model, path, max_epochs=30, patience=3):
    self.model = model
    self.path = path
    self.max_epochs = max_epochs
    self.patience = patience

  def compile(self):
    self.model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(learning_rate=0.0005),
                  metrics=[tf.metrics.MeanAbsoluteError()])

  def load_weights(self):
    self.model.load_weights(self.path)

  def fit(self, window):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=self.patience,
                                                      mode='min')

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=self.path,
                                                    save_weights_only=True,
                                                    verbose=1)

    history = self.model.fit(window.train, epochs=self.max_epochs,
                        validation_data=window.val,
                        callbacks=[early_stopping, cp_callback])
    
    return history

In [103]:
wide_window = WindowGenerator(
    train_df=train_df, val_df=val_df, test_df=test_df,
    input_width=INPUT_WIDTH, label_width=1, shift=1,
    label_columns=['position'], input_columns=FEATURES_NAMES)

In [104]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(LSTM_INPUT_WIDTH, activation='tanh', return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(L2_REGULARIZERS)),
    tf.keras.layers.Dropout(DROPOUT),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(1)
])

# Тренировка нейросети

In [105]:
model = Model(lstm_model, CHECKPOINT_PATH, max_epochs=20, patience=3)
model.compile()

if os.path.exists(f'{CHECKPOINT_PATH}/checkpoint'):
  model.load_weights()

In [ ]:
model.fit(wide_window)

Epoch 1/20
128/768 [====>.........................] - ETA: 25:19 - loss: 0.7351 - mean_absolute_error: 0.2043

In [ ]:
@property
def test_without_shuffle(self):

  return self.make_dataset(self.test_df, shuffle=False)

WindowGenerator.test_without_shuffle = test_without_shuffle

In [ ]:
model = lstm_model

In [ ]:
model.evaluate(wide_window.test)

In [ ]:
dataset = wide_window.test_without_shuffle.map(lambda x, y: model(x)).unbatch()

In [ ]:
index = test_full_df.iloc[0].name

before = np.full(INPUT_WIDTH, 0.0)
after = np.fromiter(dataset.as_numpy_iterator(), np.float32)

position_prediction_df = pd.DataFrame(data=np.concatenate((before, after), axis=0), columns=['position'], index=test_full_df.index)

In [ ]:
dataframe_with_prediction = pd.concat([position_prediction_df, test_full_df.drop(columns=['position'])], axis=1, join="inner")

In [ ]:
#@title
from pandas import DataFrame
class Strategy:
    fee = 0.2
    # ограничение на максимальное количество открытых сделок
    max_simultaneous_open_trades_limit = 1
    # максимальное количество сделок которое было открыто одновременно
    max_simultaneous_open_trades_count = 0

    def __init__(self, max_simultaneous_open_trades_limit=1, start_balance=1000):
        self.orders = pd.DataFrame(None,
                                   columns=['type', 'open_price', 'open_datetime', 'close_price', 'close_datetime',
                                            'profit', 'quantity', 'profit_as_a_percentage', 'take_profit_percentage',
                                            'stop_loss_percentage', 'take_profit_price', 'stop_loss_price'],
                                   ).astype({
            'type': str, 'open_price': np.float32, 'open_datetime': str, 'close_price': np.float32,
            'close_datetime': str,
            'profit': np.float32, 'quantity': np.float32,
            'profit_as_a_percentage': np.float32, 'take_profit_percentage': np.float32,
            'stop_loss_percentage': np.float32, 'take_profit_price': np.float32, 'stop_loss_price': np.float32
        })

        self.max_simultaneous_open_trades_limit = max_simultaneous_open_trades_limit
        self.start_balance = start_balance
        self.balance = start_balance

    def append_indicators(self, df: DataFrame) -> DataFrame:
        return df

    def get_orders(self):
        return self.orders

    def put_order(self, type: str, price: float, quantity: float, datetime: str = None,
                  close_price: float = None, close_datetime: str = None,
                  take_profit_percentage: float = None,
                  stop_loss_percentage: float = None) -> int:

        row = {
            'type': type,
            'open_price': price,
            'open_datetime': datetime,
            'close_price': close_price,
            'close_datetime': close_datetime,
            'profit': None,
            'quantity': quantity,
            'profit_as_a_percentage': None,
            'take_profit_percentage': take_profit_percentage,
            'stop_loss_percentage': stop_loss_percentage,
            'take_profit_price': None,
            'stop_loss_price': None
        }

        if take_profit_percentage is not None:
            row['take_profit_price'] = (row['take_profit_percentage'] + 100) * price / 100

        if stop_loss_percentage is not None:
            row['stop_loss_price'] = (100 - row['stop_loss_percentage']) * price / 100

        self.orders.loc[self.orders.shape[0]] = row

        open_trades_count = len(self.get_open_orders())

        if open_trades_count > self.max_simultaneous_open_trades_count:
            self.max_simultaneous_open_trades_count = open_trades_count

        self.balance = self.balance - quantity - quantity * self.fee / 100

        return self.orders.iloc[-1].name

    def is_order_closed(self, index: int):
        if self.orders.at[index, 'close_price'] or self.orders.at[index, 'close_datetime']:
            return True
        else:
            return False

    def close_order(self, index: int, price: float, datetime: str) -> bool:

        index = int(index)

        if self.is_order_closed(index):
            return False

        self.orders.at[index, 'close_price'] = float(price)
        self.orders.at[index, 'close_datetime'] = str(datetime)

        order = self.orders.iloc[index]

        fee_for_opening_trade = order['quantity'] * self.fee / 100
        fee_for_closing_trade = order['quantity'] * self.fee / 100

        if order['type'] == 'long':

            profit_as_a_percentage_without_commission = order['close_price'] * 100 / order['open_price'] - 100

            amount_on_close_trade = (order['quantity'] * (100 + profit_as_a_percentage_without_commission) / 100)

            self.balance += amount_on_close_trade - fee_for_closing_trade

            self.orders.at[index, 'profit'] = amount_on_close_trade - order['quantity'] - fee_for_opening_trade - fee_for_closing_trade
            self.orders.at[index, 'profit_as_a_percentage'] = profit_as_a_percentage_without_commission - self.fee * 2

        elif order['type'] == 'short':

            profit_as_a_percentage_without_commission = 100 - order['close_price'] * 100 / order['open_price']

            amount_on_close_trade = (order['quantity'] * (100 + profit_as_a_percentage_without_commission) / 100)

            self.balance += amount_on_close_trade - fee_for_closing_trade

            self.orders.at[index, 'profit'] = amount_on_close_trade - order['quantity'] - fee_for_opening_trade \
                                              - fee_for_closing_trade
            self.orders.at[index, 'profit_as_a_percentage'] = profit_as_a_percentage_without_commission - self.fee * 2

        return True

    def get_open_orders(self) -> DataFrame:
        return self.orders[(self.orders['close_price'].isna()) & (pd.notna(self.orders['open_datetime']))]

    def get_closed_orders(self) -> DataFrame:
        return self.orders[pd.notna(self.orders['close_price'])]

    def get_profit(self) -> float:
        return self.orders['profit'].sum()

    def get_profit_as_a_percentage(self) -> float:
        return self.orders['profit_as_a_percentage'].sum()

    def get_opened_orders_count(self) -> int:
        return len(self.get_open_orders())

    def get_closed_orders_count(self) -> int:
        return len(self.get_closed_orders())

    def get_win_trades(self) -> DataFrame:
        closed = self.get_closed_orders()
        return closed[closed['profit_as_a_percentage'] > 0]

    def get_loss_trades(self) -> DataFrame:
        closed = self.get_closed_orders()
        return closed[closed['profit_as_a_percentage'] < 0]

    def get_win_ration(self) -> float:
        return 100 * len(self.get_win_trades()) / self.get_trades_count()

    def get_trades_count(self) -> int:
        return self.get_closed_orders_count()

    def has_opened_orders(self) -> bool:
        return len(self.get_open_orders()) > 0

    def get_max_simultaneous_open_trades_count(self):
        return self.max_simultaneous_open_trades_count

    def get_pending_orders(self):
        return self.orders[(self.orders['open_datetime'].isna()) & (pd.notna(self.orders['open_price']))]

    def tick(self, candlestick):

        open_trades = self.get_open_orders()

        if len(open_trades) > 0:

            for index, item in open_trades[candlestick['low'] < open_trades['stop_loss_price']].iterrows():
                self.close_order(index, item['stop_loss_price'], candlestick['datetime'])

            for index, item in open_trades[candlestick['high'] > open_trades['take_profit_price']].iterrows():
                self.close_order(index, item['take_profit_price'], candlestick['datetime'])

        pending_orders = self.get_pending_orders()

        if len(pending_orders) > 0:

            long_pending_orders = pending_orders[pending_orders['type'] == 'long']

            for index, item in long_pending_orders[(candlestick['low'] < long_pending_orders['open_price']) & (
                    candlestick['high'] > long_pending_orders['open_price'])].iterrows():
                self.open_order(index, candlestick['datetime'])

    def open_order(self, index: int, open_datetime: np.str):
        self.orders.at[index, 'open_datetime'] = open_datetime

    def backtest(self, df: DataFrame):
        pbar = tqdm(df.iterrows(), total=len(df), colour='green')

        for index, candlestick in pbar:
            self.tick(candlestick=candlestick)
            pbar.set_description(
                f"Strategy backtesting {index} Balance: {self.get_available_balance()} Open trades: {len(self.get_open_orders())} ")

    def close_all_open_trades(self, candlestick):

        open_trades = self.get_open_orders()

        for index, item in open_trades.iterrows():
            self.close_order(index, candlestick['close'], candlestick['datetime'])

        pending_trades = self.get_pending_orders()

        for index, item in pending_trades.iterrows():
            self.cancel_order(index)

    def get_available_balance(self) -> float:
        return self.balance

    def get_start_balance(self) -> float:
        return self.start_balance

    def cancel_order(self, index: int) -> bool:

        index = int(index)

        self.orders.drop([index], inplace=True)

        return True


In [ ]:
MAX_POSITION = dataframe_with_prediction['position'].max()
LIMIT = 50 * MAX_POSITION / 100

print(MAX_POSITION, LIMIT)

In [ ]:
class LSTMStrategy(Strategy):
    fee = 0.1
    order_id = None

    def append_indicators(self, df: DataFrame) -> DataFrame:
        #df.ta.rsi(length=self.rsi, append=True)

        df = df.fillna(0)

        df = (df.replace((np.inf, -np.inf), np.nan).dropna())

        return df

    def tick(self, candlestick):
        super().tick(candlestick)

        if self.get_opened_orders_count() < 1:
          if candlestick['position'] > LIMIT:
              self.order_id = self.put_order(
                  type='long',
                  price=candlestick['close'],
                  datetime=candlestick['datetime'],
                  quantity=500,
                  take_profit_percentage=TAKE_PROFIT_PERCENT,
                  stop_loss_percentage=STOP_LOSS_PERCENT
              )

In [ ]:
strategy = LSTMStrategy()

In [ ]:
strategy.backtest(dataframe_with_prediction)
strategy.close_all_open_trades(dataframe_with_prediction.iloc[-1])

In [ ]:
strategy.get_orders()

# Результаты тестирования стратегии

In [ ]:
if len(strategy.get_orders()):
  print(f'Start balance: {strategy.get_start_balance()}')
  print(f'Available balance: {strategy.get_available_balance()}')
  print(f'Profit as percentage: {strategy.get_profit_as_a_percentage()}')
  print(f'Profit: {strategy.get_profit()}')
  print(f'Win ratio: {strategy.get_win_ration()}')
  print(f'Max simultaneous trades count: {strategy.get_max_simultaneous_open_trades_count()}')